In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
cellrebel = pd.read_csv('CellRebel_Capacity_20230205 - 06.csv', low_memory=False)
grid = gpd.read_file('poi_1.geojson')

In [14]:
cellrebel['created_date'].sort_values()

596047    2023-02-05 00:00:00.010
552263    2023-02-05 00:00:00.082
563832    2023-02-05 00:00:00.121
609470    2023-02-05 00:00:00.121
378236    2023-02-05 00:00:00.121
                   ...           
157740    2023-02-05 23:56:00.570
279149    2023-02-05 23:56:00.570
75256     2023-02-05 23:56:00.570
96344     2023-02-05 23:56:00.570
296389    2023-02-05 23:56:00.570
Name: created_date, Length: 691260, dtype: object

In [3]:
cellrebel_gpd = gpd.GeoDataFrame(cellrebel, geometry=gpd.points_from_xy(cellrebel.longitude, cellrebel.latitude), crs=4326)
grid = grid.to_crs(4326)
grid['geometry1'] = grid['geometry']

In [4]:
points_within = gpd.sjoin(left_df= cellrebel_gpd,right_df=grid, how='left', predicate='within')

In [5]:
points_within

,mobile_operator,created_date,latitude,longitude,province,kabupaten,reference_signal_received_power,geometry,index_right,Polygon,Longitude,Latitude,geometry1
0,XL Axiata,2023-02-05 11:06:24.194,-1.571500,103.544957,NaN,NaN,-44.0,POINT (103.54496 -1.57150),NaN,NaN,NaN,NaN,None
1,XL Axiata,2023-02-05 16:03:48.829,-2.552835,110.248260,NaN,NaN,-44.0,POINT (110.24826 -2.55284),NaN,NaN,NaN,NaN,None
2,XL Axiata,2023-02-05 10:02:44.701,3.567868,98.861605,NaN,NaN,-78.0,POINT (98.86161 3.56787),NaN,NaN,NaN,NaN,None
3,XL Axiata,2023-02-05 11:06:40.034,-1.572095,103.544234,NaN,NaN,-44.0,POINT (103.54423 -1.57209),NaN,NaN,NaN,NaN,None
4,XL Axiata,2023-02-05 10:02:50.685,-0.885335,103.273113,NaN,NaN,-130.0,POINT (103.27311 -0.88533),NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
691255,XL Axiata,2023-02-05 00:44:00.296,-6.223916,106.808754,NaN,NaN,-80.0,POINT (106.80875 -6.22392),NaN,NaN,NaN,NaN,None
691256,XL Axiata,2023-02-05 08:34:00.086,-6.022081,106.535369,NaN,NaN,-80.0,POINT (106.53537 -6.02208),NaN,NaN,NaN,NaN,None
691257,XL Axiata,2023-02-05 09:02:01.700,-7.767898,110.295008,NaN,NaN,-80.0,POINT (110.29501 -7.76790),NaN,NaN,NaN,NaN,None
691258,XL Axiata,2023-02-05 10:03:31.240,-1.620414,103.621136,NaN,NaN,-80.0,POINT (103.62114 -1.62041),NaN,NaN,NaN,NaN,None


In [6]:
points_within['index_right'].value_counts()

11.0    57
9.0     55
14.0     7
17.0     5
6.0      5
10.0     4
19.0     4
5.0      2
Name: index_right, dtype: int64

In [7]:
points_within['Polygon'].value_counts()

Jatiwaringin Raya        57
Cawang Intersection      55
Bundaran Semanggi         7
Sarinah Intersection      5
Pancoran Intersection     5
Cawang-Halim              4
Bundaran Patung Kuda      4
Kuningan 2 Toll Gate      2
Name: Polygon, dtype: int64

In [8]:
pivot = pd.pivot_table(points_within, index= ['Polygon', 'index_right'], values=['reference_signal_received_power'], aggfunc = np.mean)

In [9]:
pivot.reset_index(inplace=True, col_level=1, allow_duplicates=False)

In [10]:
pivot

,Polygon,index_right,reference_signal_received_power
0,Bundaran Patung Kuda,19.0,-87.000000
1,Bundaran Semanggi,14.0,-92.000000
2,Cawang Intersection,9.0,-95.327273
3,Cawang-Halim,10.0,-87.000000
4,Jatiwaringin Raya,11.0,-102.666667
5,Kuningan 2 Toll Gate,5.0,-93.000000
6,Pancoran Intersection,6.0,-109.000000
7,Sarinah Intersection,17.0,-66.600000


In [11]:
pivot = pivot.merge(points_within[['index_right', 'geometry1']], how='left', on='index_right')

In [12]:
pivot.to_csv('result.csv')